In [2]:
# Install dependencies
!pip install transformers datasets torch nltk rouge_score psutil gpustat

# Set environment variable to reduce memory fragmentation
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Hugging Face login
from huggingface_hub import login
login("HUUGING FACE CODE")  # Replace with your actual Hugging Face token

import time
import torch
import psutil
import gpustat
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import numpy as np
from tqdm import tqdm

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    device_map="auto",  # Use "0" for single GPU if OOM persists
    torch_dtype=torch.float16
)
model.eval()

# Load GSM8K dataset (first 100 samples)
dataset = load_dataset("gsm8k", "main")["train"].select(range(100))

# Initialize metrics
latencies, tokens_per_sec, perplexities, bleus, rouge1s, rougeLs, memories, f1s = [], [], [], [], [], [], [], []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Evaluation loop
for sample in tqdm(dataset, desc="Evaluating"):
    question = sample["question"]
    reference = sample["answer"].split("#### ")[-1].strip()
    prompt = f"Solve this math problem: {question}\nProvide the final answer as a number or concise phrase."

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_length = inputs.input_ids.size(1)

    # Clear GPU memory
    torch.cuda.empty_cache()

    # Measure latency and generate
    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,  # Reduced to minimize memory
            do_sample=False
        )
    latency = time.time() - start_time

    # Decode output
    generated = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True).strip()
    output_length = outputs[0][input_length:].size(0)

    # Latency and tokens per second
    latencies.append(latency)
    tokens_per_sec.append(output_length / latency if latency > 0 else 0)

    # Perplexity (forward pass, skip for very short sequences)
    if output_length > 2:
        with torch.no_grad():
            input_ids = outputs[:, input_length:].to("cuda")
            labels = input_ids.clone().to("cuda")
            outputs_forward = model(input_ids, labels=labels)
            perplexity = torch.exp(outputs_forward.loss).item()
        perplexities.append(perplexity)
    else:
        perplexities.append(float('inf'))

    # BLEU and ROUGE
    bleu = sentence_bleu([reference.split()], generated.split())
    bleus.append(bleu)
    rouge_scores = scorer.score(reference, generated)
    rouge1s.append(rouge_scores['rouge1'].fmeasure)
    rougeLs.append(rouge_scores['rougeL'].fmeasure)

    # Memory usage (GPU)
    gpu_stats = gpustat.new_query().gpus[0]
    memory_used = gpu_stats.memory_used / 1024  # Convert MB to GB
    memories.append(memory_used)

    # F1 Score (binary: correct or not)
    is_correct = generated == reference
    f1 = 1.0 if is_correct else 0.0
    f1s.append(f1)

    # Log memory usage for debugging
    print(f"Sample {len(latencies)}: GPU Memory Used: {memory_used:.3f} GB")

# Compute averages
avg_latency = np.mean(latencies)
avg_tps = np.mean(tokens_per_sec)
avg_perplexity = np.mean([p for p in perplexities if p != float('inf')]) if perplexities else float('inf')
avg_bleu = np.mean(bleus)
avg_rouge1 = np.mean(rouge1s)
avg_rougeL = np.mean(rougeLs)
avg_memory = np.mean(memories)
avg_f1 = np.mean(f1s)
avg_knowledge_retention = avg_f1
avg_flop_reduction = 0.0
avg_retrieval_latency = 0.0
avg_memory_reduction = 0.0
avg_query_time = avg_latency
avg_accuracy_drop = 0.0
avg_compression_ratio = 1.0

# Print results
print(f"Avg latency: {avg_latency:.3f} sec")
print(f"Tokens per sec: {avg_tps:.2f}")
print(f"Avg perplexity: {avg_perplexity:.2f}")
print(f"BLEU Score: {avg_bleu:.3f}")
print(f"ROUGE-1 Score: {avg_rouge1:.3f}")
print(f"ROUGE-L Score: {avg_rougeL:.3f}")
print(f"Memory usage (GB): {avg_memory:.3f}")
print(f"FLOP Reduction (%): {avg_flop_reduction:.2f}")
print(f"Retrieval Latency (sec): {avg_retrieval_latency:.3f}")
print(f"F1 Score: {avg_f1:.3f}")
print(f"Knowledge Retention: {avg_knowledge_retention:.3f}")
print(f"Memory Reduction (%): {avg_memory_reduction:.2f}")
print(f"Query Processing Time (sec): {avg_query_time:.3f}")
print(f"Accuracy Drop: {avg_accuracy_drop:.3f}")
print(f"Compression Ratio: {avg_compression_ratio:.2f}")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

2025-04-21 18:03:45.483793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745258625.756047      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745258625.834301      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

Sample 98: GPU Memory Used: 3.150 GB


Evaluating:  99%|█████████▉| 99/100 [00:41<00:00,  1.20it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Sample 99: GPU Memory Used: 3.166 GB


Evaluating: 100%|██████████| 100/100 [00:41<00:00,  2.40it/s]

Sample 100: GPU Memory Used: 3.148 GB
Avg latency: 0.194 sec
Tokens per sec: 15.20
Avg perplexity: 30.51
BLEU Score: 0.000
ROUGE-1 Score: 0.003
ROUGE-L Score: 0.003
Memory usage (GB): 3.148
FLOP Reduction (%): 0.00
Retrieval Latency (sec): 0.000
F1 Score: 0.000
Knowledge Retention: 0.000
Memory Reduction (%): 0.00
Query Processing Time (sec): 0.194
Accuracy Drop: 0.000
Compression Ratio: 1.00
